In [ ]:
# default_exp client

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
#hide
from private import server_vars

In [ ]:
#hide
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [ ]:
# export
import requests

from yx_motor.api import API
from yx_motor.authenticate import Authenticate
from yx_motor.jobs import Jobs

class Client:
    "Wrapper for Alteryx Server API."
    def __init__(self, 
                 base_url: str, 
                 login_email: str, 
                 login_pwd: str):
        """Initialize a yx_motor client object."""
        self.base_url = base_url
        self.api_url = f"{base_url}api/v1/"
        
        self.auth_endpoint = "authenticate"
        
        self.login_email = login_email
        self.login_pwd = login_pwd
        
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate"
        }
        
        self.api = API(api_url=self.api_url, 
                       headers=self.headers)
        
        self.authenticate = Authenticate(self.api)
        self.jobs = Jobs(self.api)
        
        self.authenticate.authenticate(login_email=self.login_email, 
                                       login_pwd=self.login_pwd)

    def get_users(self):
        response = self.api.get("users")
        return response


In [ ]:
from nbdev.showdoc import *
show_doc(Client.__init__)

<h4 id="Client.__init__" class="doc_header"><code>Client.__init__</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>Client.__init__</code>(**`base_url`**:`str`, **`login_email`**:`str`, **`login_pwd`**:`str`)

Initialize a yx_motor client object.

In [ ]:
motor = Client(base_url,
              login_email, 
              login_pwd)

In [ ]:
#hide
from unittest.mock import Mock
motor.api = Mock()
motor.api.is_authenticated = True

In [ ]:
motor.api.is_authenticated

True

In [ ]:
test = motor.jobs.get_job()

In [ ]:
test.json()

{'jobs': [{'jobId': '446d6653-70a8-4010-b45d-875b27c2ce4b',
   'workerId': '3b0d8827-c3a9-4e1d-ba77-304d339c71a6',
   'scheduleId': '8fad86ed-2f5e-4b99-9a5d-f45497e51635',
   'executionOrdinal': 1,
   'queuedDate': '2020-04-10T21:19:38.164Z',
   'scheduledStartDate': '2020-04-10T21:19:38.164Z',
   'actualStartDate': '2020-04-10T21:19:39.399Z',
   'completionDate': '2020-04-10T21:19:41.836Z',
   'runTime': 2,
   'status': 'completed',
   'result': 'success',
   'siteId': '2794a094-abf0-4adf-b74c-ba0f41011d17',
   'creationDate': '2020-04-10T21:19:38.153Z',
   'lastUpdate': '2020-04-10T21:19:41.829Z',
   'assetVersion': 1,
   'retryCount': 0,
   'notes': None,
   'priority': 50,
   'jobNo': 9,
   'name': 'mssql_take2.yxmd',
   'userId': '3ec0bcbd-f7e2-413b-93c0-166ee2aee5d8',
   'assetId': '23e82706-46aa-4fd2-a2a0-3992f261a02b',
   'type': 'immediate',
   'frequencyInterval': 'manual',
   'outputs': []},
  {'jobId': '755fbb0a-cc93-4ff8-8108-4eb567108fba',
   'workerId': '3b0d8827-c3a9-4e

In [ ]:
response.json()

{'asset': {'created': '2020-04-10T15:04:53.091Z',
  'versionCreated': '2020-04-10T15:04:53.091Z',
  'folderType': None,
  'links': None,
  'metadata': {'loadId': '8c08fa83-4ad5-4caa-82e3-ec002d52d99b',
   'tstamp': 1557969510,
   'extension': 'xlsx',
   'dataSource': {'id': '3e7399c5-e237-4eb0-815c-36a749c8a371',
    'name': 'ayxFile',
    'type': 'default',
    'state': 'standalone',
    'ownerId': '3ec0bcbd-f7e2-413b-93c0-166ee2aee5d8',
    'primaryId': None,
    'parameters': {'path': '\\testPath'},
    'technology': 'File',
    'description': 'Seeded datasource for File',
    'creationDate': '2019-03-26T00:00:00.000Z',
    'sharedWithIds': {'userIds': ['26e0d5a7-a53e-43ba-8de5-e505222c6e6d',
      '3c3fff1f-153f-40e1-800f-8b9ecc92de52',
      '7d81f8d2-9f18-4027-89e2-19082a73cb0a',
      'e601dc87-40f6-44d1-9aee-c65ba435b612'],
     'groupIds': []},
    'connectionCount': 1,
    'onlyOwnerCanShare': False,
    'technologyVersion': None,
    'usedInWorkflowsCount': 0},
   'parsedInf

# Jesse and JP testing stuff below (remove/breakout later)

## VFS Router API Investigation

In [ ]:
# Gets the payload of recently viewed files for a given user
response = motor.api.get("files/recentlyViewed")

In [ ]:
response

<Response [200]>

### Recently Viewed Files Response Object

- Keys at top level
 - members  (what is this exactly?  Looks like it has access control stuff for file)
 - totalCount: integer, exactly what it sounds like
 - assets: array of what appears to be vfs "asset/file" objects.  

In [ ]:
response.json().keys()

dict_keys(['members', 'totalCount', 'assets'])

In [ ]:
asset_list = response.json()['assets']

### VFS File object investigation

- fileName: exactly what you think, but no path
- path: full path, weird in a vfs as Jesse has pointed out
- folderType: What is that and what are its implications?
- version: integer
- maxVersion: Latest version.  interesting, should investigate if there are different uuids for different version or same uuid?????
- uuid: very important, core to all things workflow, vfs, etc.  all workflows appear to be vfs assets.  
- metadata: object with critical keys regarding permissions, etc.
- activityTime: ???  last run??
- contentId: Why and what is this?  Is it useful?
- entryOwner:  What is this for?  Why so much?
- permissions: list of strings of file permission types
- assetCategory: string, what are the valid types for this?  differences in behavior???
- onlySiteAdminShares: bool: Per Jesse, this is a setting on a file to ensure only admins can share or not share a file

In [ ]:
test_asset = asset_list[1]

In [ ]:
test_asset['uuid']

'22ffa1f9-4e55-4952-af90-387d2a9b9f78'

In [ ]:
test_asset['version']

1

In [ ]:
test_asset['path']

'/users/siteadmin1/simple_questions.log'

## Download File

#### File Content

Seems like content isn't what I thought it was.  

EDIT:  No, this works, but not with Excel files.  Luckily nobody uses excel in business

In [ ]:
# Didn't return a file, but not sure.

response = motor.api.get(url='files/content', 
                         params={"id": "f4929cd7-fbfc-4093-991b-17586a216f93", 
                                 "version": 1})

Below code successfully writes the file in proper format.

In [ ]:
with open('test.yxzp', 'wb') as f:
    f.write(response.content)

## Upload File

This is pretty weird.  

Talk to Alexander Potanin.  

Currently, this API is incredibly confusing.  It isn't clear, even by observing browser traffic, how a file actually gets uploaded.

Using headers for the parameters is inconsistent with, hmm, i don't know, everything else in here.  

Latest note:

https://git.alteryx.com/s2/demo-seeder/-/blob/dev/04-UploadFilesAndDecorate.js

see that we just need to specify what appears to be a binary object as payload.  

We will need to add to the existing headers for this one, due to the api inconsistencies.

Added an optional arg (just to post) for non_default_headers.

Success!  uploaded a file with the below code.

### TODO:  The code below works, so we will want to convert into a nice clean function.

In [ ]:
blob = None

with open('test.yxzp', "rb") as f:
    blob = f.read()

In [ ]:
upload_headers = {"Content-Type": "application/json",
                    "Accept": "*/*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip,deflate",
                    "path": '/Workspaces/Public/asdf/jp_test.yxzp'
                  }

In [ ]:
response = motor.api.post(url='files', 
                          data=blob,
                          non_default_headers=upload_headers)

In [ ]:
response

<Response [200]>

In [ ]:
response.json()

{'fileName': 'jp_test.yxzp',
 'extension': 'yxzp',
 'inherits': True,
 'isHidden': False,
 'onlyOwnerShares': False,
 'path': '/Workspaces/Public/asdf/jp_test.yxzp',
 'folderType': None,
 'version': 1,
 'created': '2020-04-10T23:21:20.252Z',
 'versionCreated': '2020-04-10T23:21:20.252Z',
 'maxVersion': 1,
 'links': None,
 'location': '',
 'uuid': 'bee0b8dd-6387-4740-be36-06887e0d9921',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentId': '6a411b4e-6e57-4d27-b357-421e280aaa51',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'siteadmin1@example.com',
  'firstName': 'Seeded',
  'id': '3ec0bcbd-f7e2-413b-93c0-166ee2aee5d8',
  'lastName': 'siteadmin1',
  'name': 'Seeded siteadmin1',
  'userName': 'siteadmin1'},
 'md5Hash': '1316e622c535fcfef109f75484910f38',
 'assetCategory': 'WORKFLOW_APP',
 'onlySiteAdminSh